In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Zheryu\Anaconda3\envs\image-processing\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ifft']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### Import

In [5]:
import numpy as np
import pyfftw.interfaces.scipy_fftpack as spfft
import scipy.signal as spsignal
import skimage.transform as sktransform
import matplotlib.pyplot as plt

In [6]:
from filter_tools import *
from util import *
from pyramid_tools import *

### Load the resources

In [ ]:
video_file = "baby.avi"
magnification_factor = 20
fl = .04
fh = .4
fs = 1
attenuate_other_frequencies=False
pyr_type = "octave"
sigma = 5
temporal_filter = difference_of_iir
scale_video = .8

In [ ]:
video = load_video(video_file)

#### Resize our video

In [ ]:
video2 = np.zeros((len(video), 200, 200, 3))
for i in range(len(video)):
    video2[i] = sktransform.resize(video[i], (200,200))

In [ ]:
video = video2

In [ ]:
phase_amplify(video2, magnification_factor, fl, fh, fs, attenuate_other_frequencies=attenuate_other_frequencies, pyramid_type=pyr_type, sigma=sigma, temporal_filter=temporal_filter)

In [ ]:
def phase_amplify(video, magnification_factor, fl, fh, fs, attenuate_other_frequencies=False, pyramid_type="octave", sigma=0, temporal_filter=fir_window_bp):
    num_frames, h, w, num_channels = video.shape
    pyr_height = max_scf_pyr_height((h, w))

    if pyr_type is "octave":
        print("Using vanilla octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 4)
    elif pyr_type is "halfOctave":
        print("Using half octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 8, t_width=0.75)
    elif pyr_type is "smoothHalfOctave":
        print("Using smooth half octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=2)
    elif pyr_type is "quarterOctave":
        print("Using quarter octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=4)
    else:
        print("Invalid filter type. Specify ocatave, halfOcatave, smoothHalfOctave, or quarterOctave")
        return None

    yiq_video = np.zeros((num_frames, h, w, num_channels))
    fft_video = np.zeros((num_frames, h, w), dtype=complex64)

    for i in range(num_frames):
        yiq_video[i] = rgb2yiq(video[i])
        fft_video[i] = spfft.fftshift(spfft.fft2(yiq_video[i][:,:,0]))

    magnified_y_channel = np.zeros((num_frames, h, w), dtype=complex64)
    dc_frame_index = 0
    for i in range(1,len(filters)-1):
        print("processing level "+str(i))

        dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
        dc_frame_no_mag = dc_frame / np.abs(dc_frame)    
        dc_frame_phase = np.angle(dc_frame)

        total = np.zeros(fft_video.shape, dtype=float)
        filtered = np.zeros(fft_video.shape, dtype=complex64)

        for j in range(num_frames):
            filtered[j] = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[j]))
            total[j] = simplify_phase(np.angle(filtered[j]) - dc_frame_phase)

        print("bandpassing...")
        total = temporal_filter(total, fl/fs, fh/fs).astype(float)

        for j in range(num_frames):
            phase_of_frame = total[j]
            if sigma != 0:
                phase_of_frame = amplitude_weighted_blur(phase_of_frame, np.abs(filtered[j]), sigma)

            phase_of_frame *= magnification_factor

            if attenuate_other_frequencies:
                temp_orig = np.abs(filtered[j])*dc_frame_no_mag
            else:
                temp_orig = filtered[j]
            magnified_component = 2*filters[i]*spfft.fftshift(spfft.fft2(temp_orig*np.exp(1j*phase_of_frame)))

            magnified_y_channel[j] = magnified_y_channel[j] + magnified_component

    for i in range(num_frames):
            magnified_y_channel[i] = magnified_y_channel[i] + (fft_video[i]*(filters[-1]**2))

    out = np.zeros(yiq_video.shape)

    for i in range(num_frames):
        out_frame  = np.dstack((np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i]))), yiq_video[i,:,:,1:3]))
        out[i] = yiq2rgb(out_frame)

    return out.clip(min=0, max=1)